# Debat

## 1) File Opening 

In [1]:
# Code to import libraries
import re
import json
import pandas as pd 
import nltk

In [2]:
# Read the file
file = open(r"C:\Users\ryanj\OneDrive\Bureau\dataset\xml\compteRendu\CRSANR5L15S2017E1N001.xml", encoding = 'utf-8')

In [3]:
# Explore the file content
for text in file:
    print(text)

<?xml version='1.0' encoding='UTF-8'?>

<compteRendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">

  <uid>CRSANR5L15S2017E1N001</uid>

  <metadonnees>

    <dateSeance>20170704150000000</dateSeance>

    <dateSeanceJour>mardi 04 juillet 2017</dateSeanceJour>

    <numSeanceJour>Unique</numSeanceJour>

    <numSeance>1</numSeance>

    <typeAssemblee>AN</typeAssemblee>

    <legislature>15</legislature>

    <session>Première session extraordinaire 2017</session>

    <nomFichierJo>20171001</nomFichierJo>

    <validite>valide</validite>

    <etat>complet</etat>

    <diffusion>public</diffusion>

    <version>avant_JO</version>

    <horaireGmt>Date: Wed, 05 Jul 2017 15:27:35 GMT</horaireGmt>

    <heureGeneration>2017-07-05T17:27:35.000+02:00</heureGeneration>

    <sommaire>

      <presidentSeance id_syceron="981337">Présidence de M. François de Rugy</presidentSeance>

      <sommaire1 valeur_pts_odj="1">

        <titreStruct id_syceron="981341">

          <intitule

terms = []
rootTerm = {}
relatedTerm = []
childTerm = {}
isRootTerm = False

rootTermMatch = re.compile(r"^\s*<Terms>$")
endOfTermMatch = re.compile(r"^\s*</Term>\s*")
titleMatch = re.compile(r"^\s*<Title>.*")
descriptionMatch =re.compile(r"\s*<Description>.*")
relatedTermsMatch = re.compile(r"\s*<RelatedTerms>.*")
endOfRelatedTermMatch = re.compile(r"^\s*</RelatedTerms>\s*")
relationshipMatch = re.compile(r"\s*<Relationship>.*")



for line in file:
    if rootTermMatch.match(line) != None:       # Hits the rootTerm tag: <Term>
        isRootTerm = True

    if isRootTerm and titleMatch.match(line) != None:
        rootTerm['Title'] = re.sub(r'(<Title>)|(</Title>)', '', line).strip()    # Remove the tags and white spaces
        
    if descriptionMatch.match(line) != None:
        rootTerm['Description'] = re.sub(r'(<Description>)|(</Description>)', '', line).strip()
        
    if relatedTermsMatch.match(line) !=None:     # Hits the childTerm tag after <RelatedTerms> tag
        isRootTerm = False
        rootTerm['RelatedTerms'] = []
    
    if not isRootTerm and titleMatch.match(line):
        childTerm['Title'] = re.sub(r'(<Title>)|(</Title>)', '', line).strip()
    
    if not isRootTerm and relationshipMatch.match(line):
        childTerm['Relationship'] = re.sub(r'(<Relationship>)|(</Relationship>)', '', line).strip()        
    
    if not isRootTerm and endOfTermMatch.match(line):
        childTerm = dict(sorted(childTerm.items(), key=lambda d:d[0]))  # Sort the dictionay by key
        rootTerm['RelatedTerms'].append(childTerm)                      # Add RelatedTerms list to the RootTerm dictionary
        childTerm = {}
        
        
    if endOfRelatedTermMatch.match(line):
        isRootTerm = True
    
    if isRootTerm and endOfTermMatch.match(line):
        rootTerm = dict(sorted(rootTerm.items(), key=lambda d:d[0]))  # Sort the dictionay by key 
        terms.append(rootTerm)                                        # Add RelatedTerms list to the Main dictionary
        rootTerm = {}

from os import walk #permet de récupérer les filenames d'un path avec en paramètre de haut en bas ou de bas en haut,...

def get_files_in_directory_path(path):
    print("Obtention des filenames...")
    for (dirpath, dirnames, filenames) in walk(path):
        print("")
    print("filenames récupérés\n")
    return filenames

In [4]:
import os
all_files = os.listdir(r"C:\Users\ryanj\OneDrive\Bureau\dataset\xml\compteRendu")

In [5]:
texts = []
for fichier in all_files:
    texts.append(open(f"C:/Users/ryanj/OneDrive/Bureau/dataset/xml/compteRendu/{fichier}", 'r', encoding = 'utf-8').read())
    

In [6]:
all_files[0]

'CRSANR5L15S2017E1N001.xml'

In [7]:
print(texts[1])

<?xml version='1.0' encoding='UTF-8'?>
<compteRendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">
  <uid>CRSANR5L15S2017E1N002</uid>
  <metadonnees>
    <dateSeance>20170705150000000</dateSeance>
    <dateSeanceJour>mercredi 05 juillet 2017</dateSeanceJour>
    <numSeanceJour>Unique</numSeanceJour>
    <numSeance>2</numSeance>
    <typeAssemblee>AN</typeAssemblee>
    <legislature>15</legislature>
    <session>Première session extraordinaire 2017</session>
    <nomFichierJo>20171002</nomFichierJo>
    <validite>valide</validite>
    <etat>complet</etat>
    <diffusion>public</diffusion>
    <version>avant_JO</version>
    <horaireGmt>Date: Thu, 03 Aug 2017 08:43:14 GMT</horaireGmt>
    <heureGeneration>2017-08-03T10:43:14.000+02:00</heureGeneration>
    <sommaire>
      <presidentSeance id_syceron="982174">Présidence de M. François de Rugy</presidentSeance>
      <sommaire1 valeur_pts_odj="1">
        <titreStruct id_syceron="982178">
          <intitule>Questions au Gouve

In [8]:
len(texts)

1104

## 2) Parsing using bs4

In [9]:
pip install bs4 lxml

Note: you may need to restart the kernel to use updated packages.


In [10]:
import bs4 
import lxml

In [11]:
texts[0]

'<?xml version=\'1.0\' encoding=\'UTF-8\'?>\n<compteRendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">\n  <uid>CRSANR5L15S2017E1N001</uid>\n  <metadonnees>\n    <dateSeance>20170704150000000</dateSeance>\n    <dateSeanceJour>mardi 04 juillet 2017</dateSeanceJour>\n    <numSeanceJour>Unique</numSeanceJour>\n    <numSeance>1</numSeance>\n    <typeAssemblee>AN</typeAssemblee>\n    <legislature>15</legislature>\n    <session>Première session extraordinaire 2017</session>\n    <nomFichierJo>20171001</nomFichierJo>\n    <validite>valide</validite>\n    <etat>complet</etat>\n    <diffusion>public</diffusion>\n    <version>avant_JO</version>\n    <horaireGmt>Date: Wed, 05 Jul 2017 15:27:35 GMT</horaireGmt>\n    <heureGeneration>2017-07-05T17:27:35.000+02:00</heureGeneration>\n    <sommaire>\n      <presidentSeance id_syceron="981337">Présidence de M.\xa0François de\xa0Rugy</presidentSeance>\n      <sommaire1 valeur_pts_odj="1">\n        <titreStruct id_syceron="981341">\n        

In [12]:
# Import BeautifulSoup
from bs4 import BeautifulSoup as bs
content = []
#for i in range(len(texts)):
    with texts[0] as file:
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")

IndentationError: unexpected indent (<ipython-input-12-20c27c092c8b>, line 5)

In [13]:
%%time
from bs4 import BeautifulSoup as bs
texts2 = []
content = []
allcontent = []
allsommaire = []
list_text = []
alltext = []
df = pd.DataFrame(columns=["intitule","para","text"])
for fichier in all_files[:5]:
    with open(f"C:/Users/ryanj/OneDrive/Bureau/dataset/xml/compteRendu/{fichier}", 'r', encoding = 'utf-8') as file:
        texts2.append(open(f"C:/Users/ryanj/OneDrive/Bureau/dataset/xml/compteRendu/{fichier}", 'r', encoding = 'utf-8').read())
        # Read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
        allcontent.append(bs_content)
        #df= df.append([bs_content.find_all("intitule"),bs_content.find_all("para")])
        list_sommaire = bs_content.find_all("sommaire2")
        list_text = bs_content.find_all("text")
        alltext.append(list_text)
        allsommaire.append(list_sommaire)
        #print([bs_content.find_all("intitule"),bs_content.find_all("para")])

Wall time: 427 ms


In [14]:
print(allcontent[0])

<?xml version='1.0' encoding='UTF-8'?><html><body><compterendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">
<uid>CRSANR5L15S2017E1N001</uid>
<metadonnees>
<dateseance>20170704150000000</dateseance>
<dateseancejour>mardi 04 juillet 2017</dateseancejour>
<numseancejour>Unique</numseancejour>
<numseance>1</numseance>
<typeassemblee>AN</typeassemblee>
<legislature>15</legislature>
<session>Première session extraordinaire 2017</session>
<nomfichierjo>20171001</nomfichierjo>
<validite>valide</validite>
<etat>complet</etat>
<diffusion>public</diffusion>
<version>avant_JO</version>
<horairegmt>Date: Wed, 05 Jul 2017 15:27:35 GMT</horairegmt>
<heuregeneration>2017-07-05T17:27:35.000+02:00</heuregeneration>
<sommaire>
<presidentseance id_syceron="981337">Présidence de M. François de Rugy</presidentseance>
<sommaire1 valeur_pts_odj="1">
<titrestruct id_syceron="981341">
<intitule>Ouverture de la session extraordinaire</intitule>
</titrestruct>
</sommaire1>
<sommaire1 valeur_pts_odj=

In [15]:
list_text = allcontent[0].find_all("text")
print(list_text)

[]


In [16]:
list_intitule = allcontent[0].find_all("intitule")
print(list_intitule)

[<intitule>Ouverture de la session extraordinaire</intitule>, <intitule>Déclaration de politique générale du Gouvernement, débat et vote sur cette déclaration</intitule>, <intitule>Vote en application de l’article 49, alinéa 1<exposant>er</exposant>, de la Constitution</intitule>, <intitule>
<italique>Suspension et reprise de la séance<br/></italique>
</intitule>, <intitule>Ordre du jour de la prochaine séance</intitule>]


In [17]:
intitule1 = allcontent[3].find("intitule")

In [18]:
intitule1

<intitule>Prorogation de l’état d’urgence</intitule>

In [19]:
dfintitule1 = pd.DataFrame(allcontent[3].find("intitule"),columns=["intitule"])

In [20]:
dfintitule1

,intitule
0,Prorogation de l’état d’urgence


## Cleaning

In [21]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [22]:
dfintitule2 = pd.DataFrame(allcontent[3].find_all("intitule"),columns=["intitule"])
list_intitule = list(dfintitule2["intitule"])
list_intitule

[<intitule>Prorogation de l’état d’urgence</intitule>,
 <intitule>Discussion des articles <italique>(suite)</italique></intitule>,
 <intitule>Après l’article 2</intitule>,
 <intitule>Article 3</intitule>,
 <intitule>Après l’article 3</intitule>,
 <intitule>Explications de vote</intitule>,
 <intitule>Vote sur l’ensemble</intitule>,
 <intitule>Ordre du jour de la prochaine séance</intitule>]

In [23]:
para_intitule = bs_content.find_all("intitule")
para_intitule

[<intitule>Renforcement du dialogue social</intitule>, <intitule>
 <italique>Rappel au règlement<br/></italique>
 </intitule>, <intitule>Présentation</intitule>, <intitule>Motion de rejet préalable</intitule>, <intitule>Motion de renvoi en commission</intitule>, <intitule>
 <italique>Suspension et reprise de la séance<br/></italique>
 </intitule>, <intitule>Discussion générale</intitule>, <intitule>Ordre du jour de la prochaine séance</intitule>]

In [24]:
cleanhtml(str(list(dfintitule2["intitule"])[0]))

'Prorogation de l’état d’urgence'

In [25]:
list_clean = [cleanhtml(str(raw_html)).replace('\xa03','').replace('\xa02','') for raw_html in list_intitule]
list_clean

['Prorogation de l’état d’urgence',
 'Discussion des articles (suite)',
 'Après l’article',
 'Article',
 'Après l’article',
 'Explications de vote',
 'Vote sur l’ensemble',
 'Ordre du jour de la prochaine séance']

In [26]:
#import html 
#html.unescape()

In [27]:
dfpara1 = pd.DataFrame(allcontent[0].find("para"),columns=["para"])
dfpara1

,para
0,"M. Edouard Philippe, Premier ministre"


In [28]:
para_id = bs_content.find("para")
para_id

<para id_syceron="983604"></para>

In [29]:
nextpara = para1.nextsiblings
nextpara

NameError: name 'para1' is not defined

In [30]:
test = allcontent[3].find_all("para")

SyntaxError: can't assign to function call (<ipython-input-30-001a79aca3a6>, line 1)

In [32]:
dfpara = pd.DataFrame(allcontent[3].find_all("para"),columns=["para"])
dfpara

,para
0,[]
1,"[Amendement n, [o], [20]]"
2,"[M. Didier Paris, rapporteur de la commission ..."
3,"[M. Gérard Collomb, ministre d’État, ministre ..."
4,"[Amendements n, [os], [4], , , [3], , , [19], ..."
5,"[Amendements n, [os], [8], , , [9], , , [11], ..."
6,[M. Guillaume Larrivé]
7,[Mme Marietta Karamanli]
8,[M. Yves Jégo]
9,[M. Ugo Bernalicis]


In [33]:
dfpara

,para
0,[]
1,"[Amendement n, [o], [20]]"
2,"[M. Didier Paris, rapporteur de la commission ..."
3,"[M. Gérard Collomb, ministre d’État, ministre ..."
4,"[Amendements n, [os], [4], , , [3], , , [19], ..."
5,"[Amendements n, [os], [8], , , [9], , , [11], ..."
6,[M. Guillaume Larrivé]
7,[Mme Marietta Karamanli]
8,[M. Yves Jégo]
9,[M. Ugo Bernalicis]


In [34]:
previous_siblings = list(intitule1.previous_siblings)

In [35]:
print(previous_siblings)

['\n']


In [36]:
next_siblings = list(intitule1.next_siblings)

In [37]:
next_siblings

['\n']

In [38]:
range(len(list_intitule))

range(0, 8)

In [39]:
allsommaire

[[<sommaire2>
  <titrestruct id_syceron="982046">
  <intitule>Vote en application de l’article 49, alinéa 1<exposant>er</exposant>, de la Constitution</intitule>
  </titrestruct>
  <sommaire3>
  <titrestruct id_syceron="982049">
  <intitule>
  <italique>Suspension et reprise de la séance<br/></italique>
  </intitule>
  </titrestruct>
  <para id_syceron="982067">M. Edouard Philippe, Premier ministre</para>
  </sommaire3>
  </sommaire2>], [<sommaire2>
  <titrestruct id_syceron="982180">
  <intitule>Engagements du Gouvernement</intitule>
  </titrestruct>
  <para id_syceron="982182">M. Damien Abad</para>
  <para id_syceron="982193">M. Edouard Philippe, Premier ministre</para>
  </sommaire2>, <sommaire2>
  <titrestruct id_syceron="982145">
  <intitule>Situation de l’entreprise GM&amp;S</intitule>
  </titrestruct>
  <para id_syceron="982147">M. Jean-Baptiste Moreau</para>
  <para id_syceron="982154">M. Bruno Le Maire, ministre de l’économie et des                        finances</para>
  </s

In [40]:
df = pd.DataFrame(columns=["intitule","para","text"])

In [41]:
df.head()

,intitule,para,text


In [42]:
print(allcontent[0])

<?xml version='1.0' encoding='UTF-8'?><html><body><compterendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">
<uid>CRSANR5L15S2017E1N001</uid>
<metadonnees>
<dateseance>20170704150000000</dateseance>
<dateseancejour>mardi 04 juillet 2017</dateseancejour>
<numseancejour>Unique</numseancejour>
<numseance>1</numseance>
<typeassemblee>AN</typeassemblee>
<legislature>15</legislature>
<session>Première session extraordinaire 2017</session>
<nomfichierjo>20171001</nomfichierjo>
<validite>valide</validite>
<etat>complet</etat>
<diffusion>public</diffusion>
<version>avant_JO</version>
<horairegmt>Date: Wed, 05 Jul 2017 15:27:35 GMT</horairegmt>
<heuregeneration>2017-07-05T17:27:35.000+02:00</heuregeneration>
<sommaire>
<presidentseance id_syceron="981337">Présidence de M. François de Rugy</presidentseance>
<sommaire1 valeur_pts_odj="1">
<titrestruct id_syceron="981341">
<intitule>Ouverture de la session extraordinaire</intitule>
</titrestruct>
</sommaire1>
<sommaire1 valeur_pts_odj=

In [43]:
print(allcontent[0].prettify())

<?xml version='1.0' encoding='UTF-8'?>
<html>
 <body>
  <compterendu xmlns="http://schemas.assemblee-nationale.fr/referentiel">
   <uid>
    CRSANR5L15S2017E1N001
   </uid>
   <metadonnees>
    <dateseance>
     20170704150000000
    </dateseance>
    <dateseancejour>
     mardi 04 juillet 2017
    </dateseancejour>
    <numseancejour>
     Unique
    </numseancejour>
    <numseance>
     1
    </numseance>
    <typeassemblee>
     AN
    </typeassemblee>
    <legislature>
     15
    </legislature>
    <session>
     Première session extraordinaire 2017
    </session>
    <nomfichierjo>
     20171001
    </nomfichierjo>
    <validite>
     valide
    </validite>
    <etat>
     complet
    </etat>
    <diffusion>
     public
    </diffusion>
    <version>
     avant_JO
    </version>
    <horairegmt>
     Date: Wed, 05 Jul 2017 15:27:35 GMT
    </horairegmt>
    <heuregeneration>
     2017-07-05T17:27:35.000+02:00
    </heuregeneration>
    <sommaire>
     <presidentseance id_syceron=

In [44]:
intit = allcontent[0].find("intitule")
print(intit)

<intitule>Ouverture de la session extraordinaire</intitule>


In [45]:
para = allcontent[1].find("para")
print(para)

<para id_syceron="982182">M. Damien Abad</para>


In [46]:
contenu = allcontent[0].find("contenu")
print(contenu)

<contenu>
<quantiemes>
<journee>Séance du mardi 04 juillet 2017</journee>
<session>Première session extraordinaire 2017</session>
</quantiemes>
<ouvertureseance code_grammaire="OUV_SEAN_1_1" code_parole="" code_style="Présidence" id_acteur="PA332747" id_mandat="PM722798" id_nomination_oe="0" id_nomination_op="0" id_preparation="819540" id_syceron="981337" nivpoint="1" ordinal_prise="1" ordre_absolu_seance="1" sommaire="1" valeur="" valeur_ptsodj="0">
<orateurs></orateurs>
<texte>Présidence de M. François de Rugy</texte>
<paragraphe code_grammaire="OUV_SEAN_2_1" code_parole="" code_style="NORMAL" id_acteur="PA332747" id_mandat="PM722798" id_nomination_oe="-1" id_nomination_op="PM730930" id_preparation="819541" id_syceron="981338" ordinal_prise="1" ordre_absolu_seance="2" roledebat="president" sommaire="0" valeur="" valeur_ptsodj="0">
<orateurs>
<orateur>
<nom>M. le président</nom>
<id>332747</id>
<qualite></qualite>
</orateur>
</orateurs>
<texte>La séance est ouverte.</texte>
</paragrap

In [47]:
result = list(contenu.children)
print(result)

['\n', <quantiemes>
<journee>Séance du mardi 04 juillet 2017</journee>
<session>Première session extraordinaire 2017</session>
</quantiemes>, '\n', <ouvertureseance code_grammaire="OUV_SEAN_1_1" code_parole="" code_style="Présidence" id_acteur="PA332747" id_mandat="PM722798" id_nomination_oe="0" id_nomination_op="0" id_preparation="819540" id_syceron="981337" nivpoint="1" ordinal_prise="1" ordre_absolu_seance="1" sommaire="1" valeur="" valeur_ptsodj="0">
<orateurs></orateurs>
<texte>Présidence de M. François de Rugy</texte>
<paragraphe code_grammaire="OUV_SEAN_2_1" code_parole="" code_style="NORMAL" id_acteur="PA332747" id_mandat="PM722798" id_nomination_oe="-1" id_nomination_op="PM730930" id_preparation="819541" id_syceron="981338" ordinal_prise="1" ordre_absolu_seance="2" roledebat="president" sommaire="0" valeur="" valeur_ptsodj="0">
<orateurs>
<orateur>
<nom>M. le président</nom>
<id>332747</id>
<qualite></qualite>
</orateur>
</orateurs>
<texte>La séance est ouverte.</texte>
</para

In [48]:
text_debat = allcontent[0].get("texte")
print(text_debat)

None


In [49]:
len(allcontent[0].find_all("para"))

11

In [50]:
parent = para.parent
print(parent)

<sommaire2>
<titrestruct id_syceron="982180">
<intitule>Engagements du Gouvernement</intitule>
</titrestruct>
<para id_syceron="982182">M. Damien Abad</para>
<para id_syceron="982193">M. Edouard Philippe, Premier ministre</para>
</sommaire2>


In [51]:
para

<para id_syceron="982182">M. Damien Abad</para>

In [52]:
dfintitule = pd.DataFrame(para, columns=["para"])
dfintitule

,para
0,M. Damien Abad


In [53]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [54]:
dfintitule2 = pd.DataFrame(allcontent[3].find_all("intitule"),columns=["intitule"])
list_intitule = list(dfintitule2["intitule"])
list_intitule

[<intitule>Prorogation de l’état d’urgence</intitule>,
 <intitule>Discussion des articles <italique>(suite)</italique></intitule>,
 <intitule>Après l’article 2</intitule>,
 <intitule>Article 3</intitule>,
 <intitule>Après l’article 3</intitule>,
 <intitule>Explications de vote</intitule>,
 <intitule>Vote sur l’ensemble</intitule>,
 <intitule>Ordre du jour de la prochaine séance</intitule>]

In [55]:
para_id = bs_content.find_all("para")
para_id

[<para id_syceron="983604"></para>,
 <para id_syceron="983616">M. Jean-Paul Lecoq</para>,
 <para id_syceron="983622">Mme Muriel Pénicaud, ministre du travail</para>,
 <para id_syceron="983733">M. Laurent Pietraszewski, rapporteur de la commission des affaires sociales</para>,
 <para id_syceron="983780">M. Laurent Saint-Martin, suppléant M. Joël Giraud, rapporteur pour avis de la commission des finances, de l’économie générale et du contrôle budgétaire</para>,
 <para id_syceron="983920">Mme Brigitte Bourguignon, présidente de la commission des affaires sociales</para>,
 <para id_syceron="983928">M. Boris Vallaud</para>,
 <para id_syceron="983944">M. Adrien Quatennens</para>,
 <para id_syceron="984048">Mme Muriel Pénicaud, ministre</para>,
 <para id_syceron="984055">M. Fabien Di Filippo</para>,
 <para id_syceron="984059">M. Patrick Mignola</para>,
 <para id_syceron="984075">M. Francis Vercamer</para>,
 <para id_syceron="984080">M. Éric Coquerel</para>,
 <para id_syceron="984087">Mme Hugu

In [55]:
# un intitule par 

In [69]:
import time
texts2 = []
content = []
allcontent = []
allsommaire = []
#list_text = []
#alltext = []
element = []
#element
element2 = []
#element2
df = pd.DataFrame(columns=["intitule","para"])
for fichier in all_files:
    with open(f"C:/Users/ryanj/OneDrive/Bureau/dataset/xml/compteRendu/{fichier}", 'r', encoding = 'utf-8') as file:
        texts2.append(open(f"C:/Users/ryanj/OneDrive/Bureau/dataset/xml/compteRendu/{fichier}", 'r', encoding = 'utf-8').read())
        # Read each line in the file, readlines() returns a list of lines
        t0 = time.time()
        for z in fichier:
            uid = bs_content.find("uid")
            paras = bs_content.find_all("para")
            paraclean = [cleanhtml(str(raw_html)).replace('\n','').replace('n, [os],','numero') for raw_html in para]
            dateSeance = bs_content.find("dateseance")
            metadonnees = bs_content.find("metadonnees")
            paragraphe = bs_content.find_all("paragraphe")
        content = file.readlines()
        # Combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")
        allcontent.append(bs_content)
        for j in paragraphe:
            textpara = j.find("texte")
            texteclean = [cleanhtml(str(raw_html)).replace('\n','') for raw_html in textpara]
            orateurpara = j.find("nom")
            orateurid = j.find("id")
            dictio1 = {"orateur":orateurpara, "id":orateurid, "texte":texteclean,"uid":uid}
            element.append(dictio1)
        for k in paras:
            dictio2 = {"MP's": k,"uid":uid,"dateSeance":dateSeance}
            element2.append(dictio2)
        t1 =time.time()
        print(t1 - t0)

0.20201873779296875
0.15700483322143555
0.14896821975708008
0.178969144821167
0.23403120040893555
0.25699520111083984
0.34899306297302246
0.5380175113677979
0.3449668884277344
0.457000732421875
0.4419708251953125
0.45903468132019043
0.5075862407684326
0.29399847984313965
0.43003177642822266
0.3889956474304199
0.302966833114624
0.35503721237182617
0.4219658374786377
0.5590314865112305
0.44699621200561523
0.48996686935424805
0.4010300636291504
0.45096516609191895
0.6180007457733154
0.43399572372436523
0.5030014514923096
0.724999189376831
0.5689990520477295
0.18700575828552246
0.4449656009674072
0.4799964427947998
0.5230069160461426
0.37999701499938965
0.24199414253234863
0.39096546173095703
0.5060327053070068
0.45299792289733887
0.45996952056884766
0.6190311908721924
0.4580039978027344
0.4100313186645508
0.23096394538879395
0.527033805847168
0.43196630477905273
0.23397135734558105
0.16399288177490234
0.2819676399230957
0.449998140335083
0.4600331783294678
0.28202080726623535
0.6609795093

In [70]:
df = pd.DataFrame(element)
df

,orateur,id,texte,uid
0,[M. le président],[332747],[La séance est ouverte.],[CRSANR5L15S2017E1N001]
1,None,None,"[, (La séance est ouverte à quinze heures.), ]",[CRSANR5L15S2017E1N001]
2,[M. le président],[332747],[En application des articles 29 et 30 de la Co...,[CRSANR5L15S2017E1N001]
3,[M. le président],[332747],[L’ordre du jour appelle la déclaration de pol...,[CRSANR5L15S2017E1N001]
4,[M. Edouard Philippe],[345619],"[Monsieur le président, mesdames, messieurs le...",[CRSANR5L15S2017E1N001]
...,...,...,...,...
575478,None,None,"[, (L’amendement no, , , , 2670 n’est pas ado...",[CRSANR5L15S2021O1N070]
575479,[M. le président],[1874],[La suite de la discussion budgétaire est renv...,[CRSANR5L15S2021O1N070]
575480,[M. le président],[1874],"[Prochaine séance, demain, à quinze heures :, ...",[CRSANR5L15S2021O1N070]
575481,None,None,"[, (La séance est levée , , le mardi 10, , , ...",[CRSANR5L15S2021O1N070]


In [71]:
df.to_csv('Dataset1.csv', index=False) 

In [72]:
df2 = pd.DataFrame(element2)
df2

,MP's,uid,dateSeance
0,"[M. Edouard Philippe, Premier ministre]",[CRSANR5L15S2017E1N001],[20170704150000000]
1,[M. Richard Ferrand],[CRSANR5L15S2017E1N001],[20170704150000000]
2,[M. Christian Jacob],[CRSANR5L15S2017E1N001],[20170704150000000]
3,[M. Marc Fesneau],[CRSANR5L15S2017E1N001],[20170704150000000]
4,[M. Stéphane Demilly],[CRSANR5L15S2017E1N001],[20170704150000000]
...,...,...,...
45919,"[M. Éric Woerth, président de la commission de...",[CRSANR5L15S2021O1N070],[20201109210000000]
45920,"[Amendements n, [os], [ 2514], , , [ 2516], , ...",[CRSANR5L15S2021O1N070],[20201109210000000]
45921,"[Amendements n, [os], [ 2268], , , [ 2951], , ...",[CRSANR5L15S2021O1N070],[20201109210000000]
45922,"[M. Paul-André Colombani, rapporteur pour avis...",[CRSANR5L15S2021O1N070],[20201109210000000]


In [73]:
df2.to_csv('Dataset2.csv', index=False) 

In [ ]:
list(bs_content.find_all("intitule"))

In [ ]:
df.head()

In [ ]:
df.values

In [56]:
type(bs_content)

bs4.BeautifulSoup

In [57]:
type(bs_content.find_all("intitule"))

bs4.element.ResultSet

In [58]:
type(list(bs_content.find_all("intitule")))

list

In [59]:
#Para > Intitule 
#Intitule, Para, Id Para, Texte, Orateur

In [60]:
texte = bs_content.find_all("texte")
texte

[<texte>Présidence de M. François de Rugy</texte>,
 <texte>La séance est ouverte.</texte>,
 <texte>
 <italique>(La séance est ouverte à seize heures.)</italique>
 </texte>,
 <texte>Renforcement du dialogue social</texte>,
 <texte>
 <italique>Discussion, après engagement de la procédure accélérée, d’un projet de loi</italique>
 </texte>,
 <texte>L’ordre du jour appelle la discussion, après engagement de la procédure accélérée, du projet de loi d’habilitation à prendre par ordonnances les mesures pour le renforcement du dialogue social (n<exposant>os </exposant>4, 19, 18).<br/>Avant de donner la parole à Mme la ministre du travail, je tiens à faire une mise au point sur la procédure de vote par scrutin public, dont il pourrait être fait usage dans le cadre du présent ou d’un futur débat. Celle-ci doit être demandée par les présidents de groupe ou par les représentants mandatés par ces derniers. Je rappellerai la procédure lorsque de tels scrutins auront lieu, pour que chacun ait bien le 

In [61]:
orateur = bs_content.find_all("orateur")
orateur

[<orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. Alexis Corbière</nom>
 <id>721210</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. Jean-Paul Lecoq</nom>
 <id>335612</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. Jean-Paul Lecoq</nom>
 <id>335612</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. Jean-Paul Lecoq</nom>
 <id>335612</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. le président</nom>
 <id>332747</id>
 <qualite></qualite>
 </orateur>, <orateur>
 <nom>M. Jean-Paul Lecoq</nom>
 <id>335612</id>
 <qualite></qualite>
 </orateur>, <orat

In [62]:
paragraphe = bs_content.find_all("paragraphe")
paragraphe[0]

<paragraphe code_grammaire="OUV_SEAN_2_1" code_parole="" code_style="NORMAL" id_acteur="PA332747" id_mandat="PM722798" id_nomination_oe="-1" id_nomination_op="PM730930" id_preparation="821320" id_syceron="983600" ordinal_prise="1" ordre_absolu_seance="2" roledebat="president" sommaire="0" valeur="" valeur_ptsodj="0">
<orateurs>
<orateur>
<nom>M. le président</nom>
<id>332747</id>
<qualite></qualite>
</orateur>
</orateurs>
<texte>La séance est ouverte.</texte>
</paragraphe>

In [63]:
textepara0 = paragraphe[0].find("texte")
textepara0

<texte>La séance est ouverte.</texte>

In [64]:
textepara0 = paragraphe[0].find("nom")
textepara0

<nom>M. le président</nom>

In [65]:
textepara0 = paragraphe[0].find("id")
textepara0

<id>332747</id>

In [67]:
element5 = []
uid = bs_content.find("uid")
uidclean = [cleanhtml(str(raw_html)) for raw_html in uid]
for i in paragraphe:
    textpara = i.find("texte")
    texteclean = [cleanhtml(str(raw_html)) for raw_html in textpara]
    orateurpara = i.find("nom")
    orateurid = i.find("id")
    dictio = {"orateur":orateurpara, "id":orateurid, "texte":texteclean,"uid":uidclean}
    element5.append(dictio)
#element

SyntaxError: invalid syntax (<ipython-input-67-6ca2152488b8>, line 9)

In [415]:
dftest = pd.DataFrame(element5)

In [416]:
dftest

,orateur,id,texte,uid
0,[M. le président],[332747],[La séance est ouverte.],[CRSANR5L15S2017E1N003]
1,None,None,"[\n, (La séance est ouverte à quinze heures.),...",[CRSANR5L15S2017E1N003]
2,[M. le président],[332747],[L’ordre du jour appelle les questions au Gouv...,[CRSANR5L15S2017E1N003]
3,[M. le président],[332747],"[La parole est à M. Damien Abad, pour le group...",[CRSANR5L15S2017E1N003]
4,[M. Damien Abad],[605036],"[Monsieur le président, avant de poser ma ques...",[CRSANR5L15S2017E1N003]
...,...,...,...,...
148,[M. le président],[332747],[La séance des questions au Gouvernement est t...,[CRSANR5L15S2017E1N003]
149,[M. le président],[332747],[En application de l’article L.O. 181 du code ...,[CRSANR5L15S2017E1N003]
150,[M. le président],[332747],"[Prochaine séance, demain, à neuf heures trent...",[CRSANR5L15S2017E1N003]
151,None,None,"[\n, (La séance est levée à seize heures dix.)...",[CRSANR5L15S2017E1N003]


In [293]:
# + uid 
#uid + dateseance + para 

In [294]:
uid = bs_content.find("uid")
uid

<uid>CRSANR5L15S2017E1N005</uid>

In [295]:
para1 = bs_content.find_all("para")
para1

[<para id_syceron="983604"></para>,
 <para id_syceron="983616">M. Jean-Paul Lecoq</para>,
 <para id_syceron="983622">Mme Muriel Pénicaud, ministre du travail</para>,
 <para id_syceron="983733">M. Laurent Pietraszewski, rapporteur de la commission des affaires sociales</para>,
 <para id_syceron="983780">M. Laurent Saint-Martin, suppléant M. Joël Giraud, rapporteur pour avis de la commission des finances, de l’économie générale et du contrôle budgétaire</para>,
 <para id_syceron="983920">Mme Brigitte Bourguignon, présidente de la commission des affaires sociales</para>,
 <para id_syceron="983928">M. Boris Vallaud</para>,
 <para id_syceron="983944">M. Adrien Quatennens</para>,
 <para id_syceron="984048">Mme Muriel Pénicaud, ministre</para>,
 <para id_syceron="984055">M. Fabien Di Filippo</para>,
 <para id_syceron="984059">M. Patrick Mignola</para>,
 <para id_syceron="984075">M. Francis Vercamer</para>,
 <para id_syceron="984080">M. Éric Coquerel</para>,
 <para id_syceron="984087">Mme Hugu

In [296]:
dateSeance = bs_content.find("dateSeanceJour")
dateSeance

In [297]:
metadonnees = bs_content.find_all("metadonnees")

In [298]:
type(metadonnees)

bs4.element.ResultSet

In [299]:
metadonnees

[<metadonnees>
 <dateseance>20170710160000000</dateseance>
 <dateseancejour>lundi 10 juillet 2017</dateseancejour>
 <numseancejour>1</numseancejour>
 <numseance>5</numseance>
 <typeassemblee>AN</typeassemblee>
 <legislature>15</legislature>
 <session>Première session extraordinaire 2017</session>
 <nomfichierjo>20171005</nomfichierjo>
 <validite>valide</validite>
 <etat>complet</etat>
 <diffusion>public</diffusion>
 <version>avant_JO</version>
 <horairegmt>Date: Wed, 12 Jul 2017 15:42:08 GMT</horairegmt>
 <heuregeneration>2017-07-12T17:42:08.000+02:00</heuregeneration>
 <sommaire>
 <presidentseance id_syceron="983599">Présidence de M. François de Rugy</presidentseance>
 <sommaire1 valeur_pts_odj="1">
 <titrestruct id_syceron="983603">
 <intitule>Renforcement du dialogue social</intitule>
 </titrestruct>
 <para id_syceron="983604"></para>
 </sommaire1>
 <sommaire3>
 <titrestruct id_syceron="983614">
 <intitule>
 <italique>Rappel au règlement<br/></italique>
 </intitule>
 </titrestruct>


In [300]:
metadonnees = bs_content.find_all("metadonnees")

In [301]:
bs_content.find_all("para")

[<para id_syceron="983604"></para>,
 <para id_syceron="983616">M. Jean-Paul Lecoq</para>,
 <para id_syceron="983622">Mme Muriel Pénicaud, ministre du travail</para>,
 <para id_syceron="983733">M. Laurent Pietraszewski, rapporteur de la commission des affaires sociales</para>,
 <para id_syceron="983780">M. Laurent Saint-Martin, suppléant M. Joël Giraud, rapporteur pour avis de la commission des finances, de l’économie générale et du contrôle budgétaire</para>,
 <para id_syceron="983920">Mme Brigitte Bourguignon, présidente de la commission des affaires sociales</para>,
 <para id_syceron="983928">M. Boris Vallaud</para>,
 <para id_syceron="983944">M. Adrien Quatennens</para>,
 <para id_syceron="984048">Mme Muriel Pénicaud, ministre</para>,
 <para id_syceron="984055">M. Fabien Di Filippo</para>,
 <para id_syceron="984059">M. Patrick Mignola</para>,
 <para id_syceron="984075">M. Francis Vercamer</para>,
 <para id_syceron="984080">M. Éric Coquerel</para>,
 <para id_syceron="984087">Mme Hugu

In [302]:
bs_content.find("dateseance")

<dateseance>20170710160000000</dateseance>

In [399]:
elementtest2 = []

paras = bs_content.find_all("para")
paraclean = [cleanhtml(str(raw_html)) for raw_html in paras]
for i in paras:
    uid = bs_content.find("uid")
    dateSeance = bs_content.find("dateseance")
    dictio = {"MP's": i,"uid":uid,"dateSeance":dateSeance}
    elementtest2.append(dictio)
    elementtestclean = [cleanhtml(str(raw_html)) for raw_html in elementtest2]
#element2

In [400]:
#list_clean = [cleanhtml(str(raw_html)).replace('\xa03','').replace('\xa02','') for raw_html in list_intitule]

In [401]:
dftest2 = pd.DataFrame(elementtest2)

In [402]:
dftest2

,MP's,uid,dateSeance
0,[],[CRSANR5L15S2017E1N003],[20170706093000000]
1,"[M. Gérard Collomb, ministre d’État, ministre ...",[CRSANR5L15S2017E1N003],[20170706093000000]
2,"[M. Didier Paris, rapporteur de la commission ...",[CRSANR5L15S2017E1N003],[20170706093000000]
3,[M. Ugo Bernalicis],[CRSANR5L15S2017E1N003],[20170706093000000]
4,"[M. Gérard Collomb, ministre d’État]",[CRSANR5L15S2017E1N003],[20170706093000000]
5,[Mme Naïma Moutchou],[CRSANR5L15S2017E1N003],[20170706093000000]
6,[M. Fabien Di Filippo],[CRSANR5L15S2017E1N003],[20170706093000000]
7,[Mme Laurence Vichnievsky],[CRSANR5L15S2017E1N003],[20170706093000000]
8,[M. Jean-Christophe Lagarde],[CRSANR5L15S2017E1N003],[20170706093000000]
9,[M. Olivier Dussopt],[CRSANR5L15S2017E1N003],[20170706093000000]


In [319]:
metadonnees = bs_content.find("metadonnees")
metadonnees

<metadonnees>
<dateseance>20170710160000000</dateseance>
<dateseancejour>lundi 10 juillet 2017</dateseancejour>
<numseancejour>1</numseancejour>
<numseance>5</numseance>
<typeassemblee>AN</typeassemblee>
<legislature>15</legislature>
<session>Première session extraordinaire 2017</session>
<nomfichierjo>20171005</nomfichierjo>
<validite>valide</validite>
<etat>complet</etat>
<diffusion>public</diffusion>
<version>avant_JO</version>
<horairegmt>Date: Wed, 12 Jul 2017 15:42:08 GMT</horairegmt>
<heuregeneration>2017-07-12T17:42:08.000+02:00</heuregeneration>
<sommaire>
<presidentseance id_syceron="983599">Présidence de M. François de Rugy</presidentseance>
<sommaire1 valeur_pts_odj="1">
<titrestruct id_syceron="983603">
<intitule>Renforcement du dialogue social</intitule>
</titrestruct>
<para id_syceron="983604"></para>
</sommaire1>
<sommaire3>
<titrestruct id_syceron="983614">
<intitule>
<italique>Rappel au règlement<br/></italique>
</intitule>
</titrestruct>
<para id_syceron="983616">M. 

In [320]:
for i in metadonnees:
    uid = i.find("uid")
    dateSeance = i.find("dateseance")
    dictio = {"uid":uid,"dateSeance":dateSeance}